In [7]:
!pip install torch torch_geometric torch_sparse torch_scatter torch_cluster -q


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [25]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, SAGEConv, GATConv
from sklearn.metrics import accuracy_score, f1_score


In [26]:
dataset = Planetoid(root='data/Cora', name='Cora')
data = dataset[0]

print("Number of nodes:", data.num_nodes)
print("Number of features:", data.num_node_features)
print("Number of classes:", dataset.num_classes)


Processing...


Number of nodes: 2708
Number of features: 1433
Number of classes: 7


Done!


In [27]:
# --- MLP ---
class MLP(torch.nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super().__init__()
        self.fc1 = torch.nn.Linear(in_feats, hidden_feats)
        self.fc2 = torch.nn.Linear(hidden_feats, out_feats)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# --- GCN ---
class GCN(torch.nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super().__init__()
        self.conv1 = GCNConv(in_feats, hidden_feats)
        self.conv2 = GCNConv(hidden_feats, out_feats)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# --- GraphSAGE ---
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super().__init__()
        self.sage1 = SAGEConv(in_feats, hidden_feats)
        self.sage2 = SAGEConv(hidden_feats, out_feats)

    def forward(self, x, edge_index):
        x = F.relu(self.sage1(x, edge_index))
        x = self.sage2(x, edge_index)
        return x

# --- GAT ---
class GAT(torch.nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats, heads=8):
        super().__init__()
        self.gat1 = GATConv(in_feats, hidden_feats, heads=heads, dropout=0.6)
        self.gat2 = GATConv(hidden_feats*heads, out_feats, heads=1, concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.elu(self.gat1(x, edge_index))
        x = self.gat2(x, edge_index)
        return x


In [28]:
mlp_model = MLP(dataset.num_node_features, 16, dataset.num_classes)
gcn_model = GCN(dataset.num_node_features, 16, dataset.num_classes)
sage_model = GraphSAGE(dataset.num_node_features, 16, dataset.num_classes)
gat_model = GAT(dataset.num_node_features, 8, dataset.num_classes)  # heads=8

# Optimizers
mlp_optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.01, weight_decay=5e-4)
gcn_optimizer = torch.optim.Adam(gcn_model.parameters(), lr=0.01, weight_decay=5e-4)
sage_optimizer = torch.optim.Adam(sage_model.parameters(), lr=0.01, weight_decay=5e-4)
gat_optimizer = torch.optim.Adam(gat_model.parameters(), lr=0.005, weight_decay=5e-4)  # GAT often uses smaller LR

# Loss
criterion = torch.nn.CrossEntropyLoss()


In [29]:
# --- MLP training ---
def train_mlp(model, data, optimizer, criterion, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x)  # MLP does not use edge_index
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        if epoch % 50 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# --- GNN training ---
def train_gnn(model, data, optimizer, criterion, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        if epoch % 50 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


In [30]:
@torch.no_grad()
def evaluate_mlp(model, data):
    model.eval()
    logits = model(data.x)
    pred = logits.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    f1 = f1_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu(), average='macro')
    return acc, f1

@torch.no_grad()
def evaluate_gnn(model, data):
    model.eval()
    logits = model(data.x, data.edge_index)
    pred = logits.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    f1 = f1_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu(), average='macro')
    return acc, f1


In [31]:
# --- MLP ---
print("Training MLP...")
train_mlp(mlp_model, data, mlp_optimizer, criterion)
acc, f1 = evaluate_mlp(mlp_model, data)
print(f"MLP -> Accuracy: {acc:.4f}, F1: {f1:.4f}\n")

# --- GCN ---
print("Training GCN...")
train_gnn(gcn_model, data, gcn_optimizer, criterion)
acc, f1 = evaluate_gnn(gcn_model, data)
print(f"GCN -> Accuracy: {acc:.4f}, F1: {f1:.4f}\n")

# --- GraphSAGE ---
print("Training GraphSAGE...")
train_gnn(sage_model, data, sage_optimizer, criterion)
acc, f1 = evaluate_gnn(sage_model, data)
print(f"GraphSAGE -> Accuracy: {acc:.4f}, F1: {f1:.4f}\n")

# --- GAT ---
print("Training GAT...")
train_gnn(gat_model, data, gat_optimizer, criterion)
acc, f1 = evaluate_gnn(gat_model, data)
print(f"GAT -> Accuracy: {acc:.4f}, F1: {f1:.4f}\n")


Training MLP...
Epoch 0, Loss: 1.9553
Epoch 50, Loss: 0.0073
Epoch 100, Loss: 0.0082
Epoch 150, Loss: 0.0072
MLP -> Accuracy: 0.5600, F1: 0.5523

Training GCN...
Epoch 0, Loss: 1.9561
Epoch 50, Loss: 0.0126
Epoch 100, Loss: 0.0154
Epoch 150, Loss: 0.0120
GCN -> Accuracy: 0.8050, F1: 0.7978

Training GraphSAGE...
Epoch 0, Loss: 1.9524
Epoch 50, Loss: 0.0012
Epoch 100, Loss: 0.0050
Epoch 150, Loss: 0.0042
GraphSAGE -> Accuracy: 0.7810, F1: 0.7706

Training GAT...
Epoch 0, Loss: 1.9526
Epoch 50, Loss: 0.3669
Epoch 100, Loss: 0.2924
Epoch 150, Loss: 0.3452
GAT -> Accuracy: 0.7940, F1: 0.7886

